# Times of India Article Display Interface

In [1]:
from toidisplay import display_requested_articles, load_metadata
import pandas as pd

## Define Input functions:

If you have a list or custom script to output a list of articles, a custom input function to run after display of each article (say, a coding assignment), or  

In [2]:
# Add list-defining function here:  
# The provided sample takes an input csv, an output csv, and outputs a list of articles in the input list but not in the output list

input_list = pd.read_csv('sample_input.csv', index_col=0).index.to_list()
output_csv = 'article_output.csv'

## Define Input and Save functions (Optional)
If you would like to have a custom input function run after display (say, a coding assignment), and if you would like to save the results of the input function, add them in the cell below. Define them as linked_function() and save_function() respectively. If linked_function remains undefined, the page will default to a "Press enter to continue" dialog box with no save function.

In [3]:
# The provided samples ask for a binary classification of an article, and save the output to the sample_output file 


def binary_categorize(article):
    
    category_judgment = ''
    while type(category_judgment) != bool:
        category_judgment = input("Does this article fit the category? (y or 1 = Yes, n or 0 = No)")
        if category_judgment == 'y' or category_judgment == '1':
            category_judgment = True
        elif category_judgment == 'n' or category_judgment == '0':
            category_judgment = False
        else:
            print("I'm not sure how to interpret that.")
    
    return category_judgment
    
save_location = 'sample_output.csv'

## Display Articles

In [4]:
display_requested_articles(display_list=input_list, linked_function=binary_categorize, chunk_size=15, append_mode=True, save_location=output_csv)

Loading TOI metadata...


KeyboardInterrupt: 

In [5]:
TOI_METADATA = load_metadata()

Loading TOI metadata...



done 



In [6]:
md = TOI_METADATA

In [8]:
md[md.pdf_zip.isnull()]

,headline,pub_date,start_page,url,zipped_xml_loc,txt_zip,txt_file,note,objecttypes,pdf_zip,pdf_file
record_id,,,,,,,,,,,
234435263,ANOTHER LANGWORTHY CASE,1888-02-21,7,http://search.proquest.com/docview/234435263/,../ZIP/TimesOfIndia_20170928210304_00017.zip/2...,1888.zip,234435263another_langworthy_case.txt,NaN,feature;article,NaN,NaN
234768826,Classified Ad 2 -- No Title,1908-12-05,1,http://search.proquest.com/docview/234768826/,../ZIP/TimesOfIndia_20170928210304_00017.zip/2...,1908.zip,234768826classified_ad_2__no_title.txt,NaN,classified advertisement;advertisement,NaN,NaN
234505162,Marriage Announcement 1 -- No Title,1888-02-21,4,http://search.proquest.com/docview/234505162/,../ZIP/TimesOfIndia_20170928210304_00017.zip/2...,1888.zip,234505162marriage_announcement_1__no_title.txt,NaN,news;marriage announcement,NaN,NaN
234861615,MUSLIMS IN BRITAIN,1908-12-05,10,http://search.proquest.com/docview/234861615/,../ZIP/TimesOfIndia_20170928210304_00017.zip/2...,1908.zip,234861615muslims_in_britain.txt,NaN,feature;article,NaN,NaN
234831894,BOMBAY HIGH COURT,1908-12-05,12,http://search.proquest.com/docview/234831894/,../ZIP/TimesOfIndia_20170928210304_00017.zip/2...,1908.zip,234831894bombay_high_court.txt,NaN,feature;article,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
848332528,Distant Dreams,1993-04-23,12,http://search.proquest.com/docview/848332528/,../ZIP/TimesOfIndia_20170928222400_00037.zip/8...,1993.zip,848332528distant_dreams.txt,NaN,letter to the editor;correspondence,NaN,NaN
1420235208,Other 30 -- No Title,2004-11-24,B1,http://search.proquest.com/docview/1420235208/,../ZIP/TimesOfIndia_20170928222400_00037.zip/1...,2004.zip,1420235208other_30__no_title.txt,NaN,general information,NaN,NaN
848331984,Masthead 4 -- No Title,1972-09-24,1,http://search.proquest.com/docview/848331984/,../ZIP/TimesOfIndia_20170928222400_00037.zip/8...,1972.zip,848331984masthead_4__no_title.txt,NaN,credit/acknowledgement,NaN,NaN
